In [19]:
import pandas as pd

df = pd.read_parquet('/home/ubuntu/Raijin/LATEST_DATA_WAV2VEC2.parquet')
df.head()

,sentence,path,filename
0,এক মাইলের কম থেকে শুরু করে বেশি দূরত্ব যা কিনা...,/home/ubuntu/bengali/data/train/10007728875760...,/home/ubuntu/bengali/data/train/10007728875760...
1,ব্রাজিলের জাতীয় কংগ্রেস সামাজিক বিবাহকে 10 বছর...,/home/ubuntu/bengali/data/train/10027935128938...,/home/ubuntu/bengali/data/train/10027935128938...
2,স্থলরেখা থেকে অনেক দূরে থাকায় মার্কিন যুক্তরাষ...,/home/ubuntu/bengali/data/train/10031075386419...,/home/ubuntu/bengali/data/train/10031075386419...
3,বর্তমানে অনেক সামি আধুনিক ব্যবসায় কাজ করে।সামি...,/home/ubuntu/bengali/data/train/10032118353482...,/home/ubuntu/bengali/data/train/10032118353482...
4,অশ্বরোহীর পাদান তার পায়ের সাপোর্টের জন্য যা অশ...,/home/ubuntu/bengali/data/train/10033964478741...,/home/ubuntu/bengali/data/train/10033964478741...


In [20]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(df['sentence'].tolist()))

In [22]:
!kenlm/build/bin/lmplz --text text.txt --arpa out.arpa -o 5 --prune 0 1 1
!kenlm/build/bin/build_binary -q 8 -b 7 -a 256 trie out.arpa out.trie.klm

=== 1/5 Counting and sorting n-grams ===
Reading /home/ubuntu/bengali/speech-to-text/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 8195431 types 196542
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:2358504 2:18123259904 3:33981116416 4:54369783808 5:79289270272
Statistics:
1 196542 D1=0.701262 D2=0.963045 D3+=1.21669
2 1743729/1899206 D1=0.802367 D2=1.16807 D3+=1.32639
3 2835867/3131703 D1=0.83147 D2=1.42044 D3+=1.54664
4 2927430/3250095 D1=0.805353 D2=1.50683 D3+=2.22252
5 2638681/2936409 D1=0.0653422 D2=1.9611 D3+=2.96163
Memory estimate for binary LM:
type     MB
probing 221 assuming -p 1.5
probing 265 assuming -r models -p 1.5
trie    112 without quantization
trie     62 assuming -q 8 -b 8 quantization 
trie     98 assuming -a 22 array pointer compression
trie     49 assuming -a 22 -q

In [12]:
!ls -lh

total 855M
-rwxrwxrwx  1 ubuntu ubuntu  55K Sep 15 16:46  Inference-Trial-0.ipynb
-rwxrwxrwx  1 ubuntu ubuntu 2.2K Sep  3 15:55  README.md
-rw-r--r--  1 ubuntu ubuntu  915 Sep  7 11:31  audio-augmentation.ipynb
-rwxrwxrwx  1 ubuntu ubuntu 9.5M Sep 15 15:32  bengali-speech-eda.ipynb
-rwxrwxrwx  1 ubuntu ubuntu 767K Sep 10 08:41  check-unknown.ipynb
-rw-r--r--  1 ubuntu ubuntu  24K Sep 13 10:12  create-fold.ipynb
-rwxrwxrwx  1 ubuntu ubuntu  13K Sep 10 02:29  explore-vocab.ipynb
-rw-r--r--  1 ubuntu ubuntu 7.6K Sep  6 04:01  inference-ariff.ipynb
-rwxrwxrwx  1 ubuntu ubuntu  32K Sep  3 15:55  inference-validation.ipynb
-rwxrwxrwx  1 ubuntu ubuntu  10K Sep  3 15:55  inference.ipynb
-rwxrwxrwx  1 ubuntu ubuntu  245 Sep 16 03:19  install-kenlm.sh
drwxr-xr-x  9 ubuntu ubuntu 4.0K Feb 11  2022  kenlm
drwxr-xr-x  3 ubuntu ubuntu 4.0K Sep 13 00:11  kenlm-4gram
drwxr-xr-x  2 ubuntu ubuntu 4.0K Sep 11 08:37  kenlm-arpa
-rw-r--r--  1 ubuntu ubuntu 133M Sep 16 03:03  kenlm-deprecated-text.txt
-rw-r

In [14]:
!head -20 ./kenlm-4gram-newdataset/out.arpa

\data\
ngram 1=196542
ngram 2=1743729
ngram 3=2835867
ngram 4=2927430

\1-grams:
-6.3277507	<unk>	0
0	<s>	-1.2403845
-1.7006247	</s>	0
-2.8978767	এক	-0.37472534
-5.34024	মাইলের	-0.14626579
-3.6946073	কম	-0.24526523
-2.604384	থেকে	-0.43365178
-3.132855	শুরু	-1.0808644
-2.7338002	করে	-0.45755637
-3.2481034	বেশি	-0.32089615
-4.302331	দূরত্ব	-0.19532079
-3.1718116	যা	-0.3034509
-3.9696317	কিনা	-0.36083865


In [16]:
import json

with open('/home/ubuntu/Raijin/full_vocab.json') as fopen:
    vocab = json.load(fopen)
    vocab = {v: k for k, v in vocab.items()}
    
vocab = [vocab[i] for i in range(len(vocab))]
vocab
len(vocab)

150

In [17]:
import kenlm
from pyctcdecode import build_ctcdecoder

lm = './kenlm-4gram-newdataset/out.arpa'
kenlm_model = lm
decoder = build_ctcdecoder(
    vocab,
    kenlm_model,
    alpha=0.5,
    beta=1.0,
)

Loading the LM will be faster if you build a binary file.
Reading /home/ubuntu/bengali/speech-to-text/kenlm-4gram-newdataset/out.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [18]:
# test model

In [ ]:
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor
) 
import json
import os

In [ ]:
import datasets
import random
import pandas as pd
import torch
from transformers import AutoTokenizer
from typing import Mapping, Tuple
import librosa
# import en_core_web_sm


import librosa

class BengaliDataset(torch.utils.data.Dataset):
    def __init__(self, df, processor):
        self.df = df
        self.processor = processor


    def __getitem__(self, idx):
        audio_path = self.df.loc[idx]['path']
        audio_array = self.read_audio(audio_path)
        
        inputs = self.processor(
            audio_array,
            sampling_rate=16000,
            return_tensors='pt'  
        )
        
        with self.processor.as_target_processor():
            labels = self.processor(self.df.loc[idx]['sentence']).input_ids
        
        return {'input_values': inputs['input_values'][0], 'labels': labels}
        
    def __len__(self):
        return len(self.df)

    def read_audio(self, mp3_path):
        target_sr = 16000  # Set the target sampling rate
        
        audio, sr = librosa.load(mp3_path, sr=None)  # Load with original sampling rate
        audio_array = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
        
        return audio_array


In [ ]:
processor = AutoProcessor.from_pretrained("/home/ubuntu/bengali/aisyah/training/mms-1b/checkpoint-16400")
model = Wav2Vec2ForCTC.from_pretrained("/home/ubuntu/bengali/aisyah/training/mms-1b/checkpoint-16400")

In [ ]:
train, test = train_test_split(df, test_size=0.02, random_state=42)

test = test.reset_index(drop=True)

In [ ]:
batch_size = 10
test_ds = BengaliDataset(test, processor)

In [ ]:
def custom_collate_fn(batch):
    input_values = [item['input_values'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_values_padded = pad_sequence(input_values, batch_first=True, padding_value=0.0)

    labels = [torch.tensor(label) for label in labels]

    return {'input_values': input_values_padded, 'labels': labels}


In [ ]:
test_loader = DataLoader(
    test_ds, batch_size=batch_size, shuffle=False, num_workers=2, collate_fn=custom_collate_fn
)

In [ ]:
model.to('cuda')
model.half()

In [ ]:
sentences = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        x = batch["input_values"]
        x = x.to("cuda", non_blocking=True)
        with torch.cuda.amp.autocast(True):
            y = model(x).logits
        y = y.detach().cpu().numpy()

        for l in y:
            out = decoder.decode_beams(l,prune_history = True)
            d_lm, lm_state, timesteps, logit_score, lm_score = out[0]
            sentences.append(d_lm)

In [ ]:
import jiwer

jiwerresult_0 = []

for x in range(len(sentences)):

    jiwer_ = jiwer.wer(sentences[x],test.iloc[x,1])

    jiwerresult_0.append(jiwer_)
    
    
avg_wer_0 = sum(jiwerresult_0)/len(sentences)
print(avg_wer_0)